## Targets from README.md
8. Down-select from possible reactions so that ~250K reactions generated as uniformly as possible across different templates. Flag the "test" set (e.g. no ligand swaps possible rxns). Include all rxns from templates with from those less than 250K/125. Take (250-all enumerated)/n_remaining samples from remaining rows if possible.

In [3]:
import pandas as pd
import numpy as np

In [4]:
n = 250000
random_state = 42

In [5]:
df = pd.read_pickle(
    'enumerated_rxns.pkl'
)

In [6]:
df.iloc[0]

rxn_uid                               MOR1_Eu2_Charge2_UHF7_swaplig_0_newlig_c1ccncc13
rxn_name                                                                          MOR1
reactants                                                                      ED01+CO
products                                                                          PR01
init_sdf                             charge=0\n     RDKit          3D\n\n  0  0  0 ...
final_sdf                            charge=0\n     RDKit          3D\n\n  0  0  0 ...
metal_inds                                                                         [2]
metal_syms                                                                        [Eu]
metal_oxs                                                                          [2]
total_charge                                                                         2
total_uhf                                                                            7
swap_remove_inds                           

In [7]:
total_swaps = 250000
out_dfs = []
test_df = df[df.possible_lig_swaps == 0]
out_dfs.append(test_df)
# 1.17 selected to get to the point where all sampled dataframes have more than the all_dfs included, but no more.
factor = 1.17
all_dfs = df[(df.possible_lig_swaps != 0) & (
    (df.possible_n_swaps) < (total_swaps / (df.rxn_df_index.unique().shape[0] - test_df.rxn_df_index.unique().shape[0]))*factor)]
out_dfs.append(all_dfs)
remaining_n = total_swaps - test_df.shape[0] - all_dfs.shape[0]
to_sample_inds = np.concatenate([test_df.rxn_df_index.unique(), all_dfs.rxn_df_index.unique()])
to_sample_df = df[~df.rxn_df_index.isin(to_sample_inds)]
sample_per_n = int(np.floor(remaining_n / to_sample_df.rxn_df_index.unique().shape[0]))
print('Remaining samples:',remaining_n,'Remaining unique reaction templates:',to_sample_df.rxn_df_index.unique().shape[0], 'Samples needed per template: ', sample_per_n)
print('Number of unique reactions that have fewer possible swaps than samples remaining: ',to_sample_df[to_sample_df.possible_n_swaps < sample_per_n].shape[0])
for rxn_ind in to_sample_df.rxn_df_index.unique():
    tdf = to_sample_df[to_sample_df.rxn_df_index == rxn_ind]
    try:
        sdf = tdf.sample(sample_per_n, random_state=random_state)
    except:
        sdf = tdf
        print(sample_per_n, tdf.shape[0], 'Actual number is lower than expected - including all from rxn', rxn_ind)
    out_dfs.append(sdf)
concat_out_df = pd.concat(out_dfs)
concat_out_df.reset_index(drop=True,inplace=True)

Remaining samples: 208872 Remaining unique reaction templates: 66 Samples needed per template:  3164
Number of unique reactions that have fewer possible swaps than samples remaining:  0
3164 2926 Actual number is lower than expected - including all from rxn 30
3164 2774 Actual number is lower than expected - including all from rxn 31
3164 2109 Actual number is lower than expected - including all from rxn 64
3164 3116 Actual number is lower than expected - including all from rxn 114


In [8]:
concat_out_df.to_pickle('../4_production/to_gen_structures_parallel.pkl')